In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('dataframes/google_places_data.csv')

# Feature engineering

In this part we will focus on:
* dealing with missing data
* normalizing numerical features
* selecting relevant columns
* adding new features